In [123]:
import pandas as pd

In [124]:
df = pd.read_csv('train.csv')
dftest = pd.read_csv('test.csv')

In [125]:
df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [126]:
dftest.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [127]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [128]:
dftest.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

### Consertando missing data (substituir pela média da variável ou mediana)

In [129]:
df['Age'].fillna(df['Age'].mean(),inplace=True)
dftest['Age'].fillna(dftest['Age'].mean(),inplace=True)

### Consertando missing data (substituir pelo mais frequente ou deletar as linhas)

In [130]:
df['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [131]:
df['Embarked'].fillna("S",inplace=True)

In [132]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

### Categorização do gênero e embarked e Pclass (Dummy Variable)

In [133]:
df['Pclass'],_ = df['Pclass'].factorize()
df['Pclass'] = df['Pclass'].astype(object)

dftest['Pclass'],_ = dftest['Pclass'].factorize()
dftest['Pclass'] = dftest['Pclass'].astype(object)

In [134]:
dummies = pd.get_dummies(df[['Embarked','Sex','Pclass']],drop_first=True)

dummiestest = pd.get_dummies(dftest[['Embarked','Sex','Pclass']],drop_first=True)

In [135]:
df = df.drop(['Embarked','Sex','Pclass','Ticket','Cabin','Name','PassengerId'],axis=1)

dftest = dftest.drop(['Embarked','Sex','Pclass','Ticket','Cabin','Name'],axis=1)

In [136]:
df = pd.concat([df,dummies],axis=1)

dftest = pd.concat([dftest,dummiestest],axis=1)

In [137]:
dftest.head()

,PassengerId,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_male,Pclass_1,Pclass_2
0,892,34.5,0,0,7.8292,1,0,1,0,0
1,893,47.0,1,0,7.0000,0,1,0,0,0
2,894,62.0,0,0,9.6875,1,0,1,1,0
3,895,27.0,0,0,8.6625,0,1,1,0,0
4,896,22.0,1,1,12.2875,0,1,0,0,0


In [139]:
X = df.drop("Survived",axis=1)
y = df.Survived


Xtest = dftest.drop("PassengerId",axis=1)

In [140]:
X.head()

,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_male,Pclass_1,Pclass_2
0,22.0,1,0,7.2500,0,1,1,0,0
1,38.0,1,0,71.2833,0,0,0,1,0
2,26.0,0,0,7.9250,0,1,0,0,0
3,35.0,1,0,53.1000,0,1,0,1,0
4,35.0,0,0,8.0500,0,1,1,0,0


In [141]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.20, random_state=1)

In [142]:
import xgboost as xgb

In [143]:
xg = xgb.XGBClassifier()

In [144]:
clf = xg.fit(X_train, y_train)

In [145]:
p = clf.predict(X_test)

In [146]:
from sklearn.metrics import classification_report, confusion_matrix

In [147]:
print(classification_report(y_test, p))

              precision    recall  f1-score   support

           0       0.77      0.93      0.84       106
           1       0.86      0.59      0.70        73

    accuracy                           0.79       179
   macro avg       0.81      0.76      0.77       179
weighted avg       0.81      0.79      0.78       179



In [148]:
confusion_matrix(y_test, p)

array([[99,  7],
       [30, 43]], dtype=int64)

### Predict no dataFrame de teste

vou usar o dataframe inteiro de treino como dados de "treino" e o dataFrame de teste como dados de "teste"

In [149]:
xgb_final = xgb.XGBClassifier()

In [150]:
model_final = xgb_final.fit(X,y)

In [151]:
ytest = model_final.predict(Xtest)

In [153]:
sub = pd.DataFrame({'PassengerId': dftest["PassengerId"], 'Survived': ytest})
print(sub.head(10))

   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         0
5          897         0
6          898         1
7          899         0
8          900         1
9          901         0


In [154]:
sub.to_csv('submissionLevy.csv', index=False)
